# Import packages

In [ ]:
# # if using google drive
# %cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class
from machine_learning.ml_methods.advanced_ml_methods import advanced_regression_utils, advanced_classification_utils, reg_feat_importance

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



# Predict num stops

## data from all sessions

In [ ]:
gc_kwargs = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()

gc_kwargs_temp = {**gc_kwargs,
                    'num_old_ff_per_row': 2,
                    'num_new_ff_per_row': 3}

gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp, monkey_name='monkey_Schro')
gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=True, traj_df_exist_in_GUAT_store_ok=True)
gas.unpack_and_reload_combined_info_back_to_self(gas.combined_info, gas.all_traj_feature_names)

# shared part with 'data from one session'
gas.process_current_and_alternative_ff_info()
more_ff_attributes = ['ff_distance', 'ff_angle', 'ff_angle_boundary', 'curv_diff']
ff_last_seen_attributes = ['last_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_last_seen', 'angle_from_monkey_now_to_monkey_when_ff_last_seen']
ff_next_seen_attributes = ['next_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_next_seen', 'angle_from_monkey_now_to_monkey_when_ff_next_seen']
gas.find_input_and_output(add_arc_info=True, add_current_curv_of_traj=True, 
                          ff_attributes=['ff_distance', 'ff_angle', 'ff_angle_boundary', 'time_since_last_vis', 'duration_of_last_vis_period']
                          + ff_last_seen_attributes)

## data from one session

In [ ]:
# gc_kwargs = helper_GUAT_vs_TAFT_class.gc_kwargs.copy()

# raw_data_folder_path = 'all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330'
# gcc = GUAT_collect_info_class.GUATCollectInfoForSession(raw_data_folder_path=raw_data_folder_path, 
#                                                         gc_kwargs=gc_kwargs, new_point_index_start=0)
# _ = gcc.streamline_process_to_collect_info_from_one_session(GUAT_w_ff_df_exists_ok=True,
#                                                             GUAT_info_exists_ok=False)

# gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs)
# gas.unpack_and_reload_combined_info_back_to_self(gcc.important_info, gcc.all_traj_feature_names)

# # shared part with 'data from all sessions'
# gas.process_current_and_alternative_ff_info()
# more_ff_attributes = ['ff_distance', 'ff_angle', 'curv_diff']
# ff_last_seen_attributes = ['last_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_last_seen', 'angle_from_monkey_now_to_monkey_when_ff_last_seen']
# ff_next_seen_attributes = ['next_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_next_seen', 'angle_from_monkey_now_to_monkey_when_ff_next_seen']
# gas.find_input_and_output(add_arc_info=True, add_current_curv_of_traj=True, 
#                           ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis', 'duration_of_last_vis_period']
#                           + ff_last_seen_attributes)
# # additional features we can try: 'time_till_next_visible'

## ML

### FIRST get data

In [ ]:
gas.furnish_with_trajectory_data = True
gas.prepare_data_for_machine_learning()
y_var_df = pd.DataFrame(gas.num_stops, columns=['num_stops'])
y_var_df['num_stops'] = y_var_df['num_stops'].astype(int)


# and for classification
y_var_df2 = y_var_df.copy()
# y_var_df2.loc[y_var_df2['num_stops'] > 2, 'num_stops'] = 2
y_var_df2.loc[y_var_df2['num_stops'] > 2, 'num_stops'] = 2
y_var_df2['num_stops'] = y_var_df2['num_stops'].astype(int)
y_var_df2['num_stops'] = y_var_df2['num_stops'] - 1  # to avoid problem during classification


### exp

In [ ]:
gas.X_all_df.shape

In [ ]:
gas.X_all_df.columns

# Regression

In [ ]:
ml_inst = ml_methods_class.MlMethods(x_var_df= gas.X_all_df,
                                     y_var_df=y_var_df)

In [ ]:
#model_names=['linreg', 'grad_boosting', 'rf']
model_names=None

ml_inst.use_ml_model_for_regression(ml_inst.x_var_df, ml_inst.y_var_df, model_names=model_names)
ml_inst.model_comparison_df

## advanced regression

In [ ]:
ml_inst.use_train_test_split(ml_inst.x_var_df, ml_inst.y_var_df)
df, best = advanced_regression_utils.use_advanced_model_for_regression(
    ml_inst.X_train, ml_inst.y_train, ml_inst.X_test, ml_inst.y_test,
    use_cv=True, cv_splits=5, verbose=True,
    save_dir="all_monkey_data/temp_ml_results/pred_num_stops",
    resume=False
)

In [ ]:
best

## feature importance

### quicker

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, AdaBoostRegressor

# Example: fit your models
dt = DecisionTreeRegressor(random_state=42).fit(ml_inst.x_var_df, ml_inst.y_var_df)
gb = GradientBoostingRegressor(random_state=42).fit(ml_inst.x_var_df, ml_inst.y_var_df)
hgb = HistGradientBoostingRegressor(random_state=42).fit(ml_inst.x_var_df, ml_inst.y_var_df)
ada = AdaBoostRegressor(random_state=42).fit(ml_inst.x_var_df, ml_inst.y_var_df)

# Collect in a dict
models = {
    "Decision Tree": dt,
    "Gradient Boosting": gb,
    "HistGradientBoosting": hgb,
    "AdaBoost": ada,
}

# Get feature importances for each model
for name, model in models.items():
    if hasattr(model, "feature_importances_"):
        importances = model.feature_importances_
        feature_importance = pd.DataFrame({
            "feature": ml_inst.x_var_df.columns,
            "importance": importances
        }).sort_values(by="importance", ascending=False)
        
        print(f"\n{name} Feature Importances:")
        print(feature_importance.head(10))  # top 10
    else:
        print(f"\n{name} does not expose feature_importances_.")


In [ ]:
feature_importance.head(20)

### in the future I can figure out...

In [ ]:

# models = {
#     "hist_gb": fitted_hist_gradient_boosting_regressor_or_pipeline,
#     "dt": fitted_decision_tree_regressor_or_pipeline,
#     "grad_boosting": fitted_gradient_boosting_regressor_or_pipeline,
#     "boosting": fitted_adaboost_or_xgboost_or_lightgbm_or_pipeline,
# }

# # For each model:
# for name, mdl in models.items():
#     try:
#         df_imp = feature_importance_table(mdl, X_valid, y_valid)  # y_valid only needed if fallback
#         print(f"\n{name} — top 10")
#         print(df_imp.head(10))
#     except Exception as e:
#         print(f"{name}: {e}")


# Classification

In [ ]:
# in case you forgot to do this first:
gas.furnish_with_trajectory_data = True
gas.prepare_data_for_machine_learning() 
y_var_df = pd.DataFrame(gas.num_stops, columns=['num_stops'])
y_var_df['num_stops'] = y_var_df['num_stops'].astype(int)


# and for classification
y_var_df2 = y_var_df.copy()
y_var_df2.loc[y_var_df2['num_stops'] > 2, 'num_stops'] = 2
y_var_df2['num_stops'] = y_var_df2['num_stops'].astype(int)
y_var_df2['num_stops'] = y_var_df2['num_stops'] - 1  # to avoid problem during classification


In [ ]:
ml_inst = ml_methods_class.MlMethods(x_var_df= gas.X_all_df,
                                     y_var_df=y_var_df2)
ml_inst.use_train_test_split(ml_inst.x_var_df, ml_inst.y_var_df)
ml_inst.use_ml_model_for_classification(ml_inst.x_var_df, ml_inst.y_var_df, model_names=model_names)


## advanced classification

In [ ]:
from machine_learning.ml_methods.advanced_ml_methods import advanced_regression_utils, advanced_classification_utils, reg_feat_importance
reload(advanced_classification_utils)

In [ ]:
ml_inst.y_test.value_counts()

In [ ]:
resume = False
tune = False

ml_inst = ml_methods_class.MlMethods(x_var_df= gas.X_all_df,
                                     y_var_df=y_var_df2)
ml_inst.use_train_test_split(ml_inst.x_var_df, ml_inst.y_var_df)
model, y_pred, model_comparison_df = advanced_classification_utils.use_advanced_model_for_classification(
    ml_inst.X_train, ml_inst.y_train, ml_inst.X_test, ml_inst.y_test,
    kfold_cv=5,
    tune=tune,                # turn tuning on/off
    n_iter=30,                # ~3k samples sweet spot
    tune_scoring="balanced_accuracy",
    checkpoint_dir="pred_num_stops/cls_runs",   # folder to save progress
    resume=resume,              # skip finished models on rerun
    n_jobs=-1,
    verbose=True
)


In [ ]:
model_comparison_df

# model's feature selection

statsmodels, logreg

Can't use it here because we are not predicting 2 classes

In [ ]:
# import statsmodels.api as sm

# X2 = sm.add_constant(ml_inst.x_var_df)  # add intercept
# logit_model = sm.Logit(ml_inst.y_var_df, X2)  # logistic regression
# result = logit_model.fit()

# print(result.summary())


## random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(ml_inst.x_var_df, ml_inst.y_var_df)

importances = rf.feature_importances_
feature_importance = pd.DataFrame({
    "feature": ml_inst.x_var_df.columns,
    "importance": importances
}).sort_values(by="importance", ascending=False)

print(feature_importance)


## grad_boosting (so that we can see feature importance)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns


# Define the model
model = GradientBoostingClassifier(
    learning_rate=0.05, max_depth=7, max_features='sqrt',
    min_samples_leaf=2, min_samples_split=7,
    n_estimators=500, subsample=0.5
)


# Fit the model
model.fit(ml_inst.x_var_df, ml_inst.y_var_df)

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for feature importances
feature_importances_df = pd.DataFrame({
    'Feature': ml_inst.x_var_df.columns,
    'Importance': feature_importances
})

# Sort the DataFrame by importance
feature_importances_df = feature_importances_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 18))
sns.barplot(x='Importance', y='Feature', data=feature_importances_df)
plt.title('Feature Importances')
plt.show()

# Determine significant features (e.g., importance > 0.01)
significant_features = feature_importances_df[feature_importances_df['Importance'] > 0.01]
print("Significant features:")
print(significant_features)

## sequential selection...? try it later

In [ ]:
# from sklearn.linear_model import LogisticRegression

# model = LogisticRegression(penalty='l1', solver='liblinear')
# model.fit(ml_inst.x_var_df, ml_inst.y_var_df)
# important = model.coef_[0] != 0
# X_new = ml_inst.x_var_df.loc[:, important]
# X_new.columns

# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.ensemble import RandomForestClassifier

# model = RandomForestClassifier()
# sfs = SequentialFeatureSelector(model, n_features_to_select=10, direction="forward")
# X_new = sfs.fit_transform(ml_inst.x_var_df, ml_inst.y_var_df.values.ravel())


# # Boolean mask of selected features
# mask = sfs.get_support()

# # Names of selected features
# selected_features = ml_inst.x_var_df.columns[mask]

# print("Selected features:")
# print(selected_features)

# Old method

In [ ]:
stop!

In [ ]:
gas.prepare_data_for_machine_learning()
gas.split_data_to_train_and_test(scaling_data=True)

#bagging = BaggingClassifier(n_estimators=200, max_features=0.9, bootstrap_features=True, bootstrap=True, random_state=42)
#gas.use_machine_learning_model_for_classification(model=bagging) 

gas.use_machine_learning_model_for_classification(model=None) #MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=1000, activation='relu', solver='adam', random_state=1))

gas.get_pred_results_df()  
gas.add_additional_info_to_plot()

In [ ]:
# get the proportion of cases where the number of stops is 1
len(gas.num_stops[gas.num_stops==1])/len(gas.num_stops)

In [ ]:
import seaborn as sns
sns.histplot(gas.num_stops)
plt.show()

### plot prediction results

In [ ]:
gas.plot_prediction_results(max_plot_to_make=5, show_reward_boundary=True, use_more_ff_inputs=True, use_more_traj_points=True,
                            predict_num_stops=True,
                            )

### show more plots

In [ ]:
current_i = 0
max_plot_to_make = 3

In [ ]:

selected_cases = np.arange(current_i, current_i+max_plot_to_make)
gas.plot_prediction_results(selected_cases=selected_cases, 
                            max_plot_to_make=max_plot_to_make, 
                            show_reward_boundary=True, 
                            use_more_ff_inputs=True, 
                            use_more_traj_points=True,
                            predict_num_stops=True)
current_i += max_plot_to_make

In [ ]:
gas.use_machine_learning_model_for_classification(model = MLPClassifier(hidden_layer_sizes=(100, 100, 100), max_iter=1000))   
gas.get_pred_results_df()  

### loop through num_ff (as hyperparam)

In [ ]:
# loop
for num_old_ff_per_row in [1, 2, 3]:
    for num_new_ff_per_row in [1, 2, 3]:
        print('num_old_ff_per_row:', num_old_ff_per_row)
        print('num_new_ff_per_row:', num_new_ff_per_row)

        gc_kwargs_temp = {**gc_kwargs,
                            'num_old_ff_per_row': num_old_ff_per_row,
                            'num_new_ff_per_row': num_new_ff_per_row}

        gas = GUAT_combine_info_class.GUATCombineInfoAcrossSessions(gc_kwargs_temp)
        gas.retrieve_or_make_combined_info(gc_kwargs_temp, combined_info_exists_ok=True, traj_df_exist_in_GUAT_store_ok=True)
        gas.unpack_and_reload_combined_info_back_to_self(gas.combined_info, gas.all_traj_feature_names)


        gas.process_current_and_alternative_ff_info()
        more_ff_attributes = ['ff_distance', 'ff_angle', 'curv_diff']
        ff_last_seen_attributes = ['last_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_last_seen', 'angle_from_monkey_now_to_monkey_when_ff_last_seen']
        ff_next_seen_attributes = ['next_seen_' + attribute for attribute in more_ff_attributes] + ['distance_from_monkey_now_to_monkey_when_ff_next_seen', 'angle_from_monkey_now_to_monkey_when_ff_next_seen']


        gas.find_input_and_output(add_arc_info=True, add_current_curv_of_traj=True, 
                                ff_attributes=['ff_distance', 'ff_angle', 'time_since_last_vis', 'time_till_next_visible', 'duration_of_last_vis_period']
                                + ff_last_seen_attributes)

        gas.prepare_data_for_machine_learning()
        gas.split_data_to_train_and_test(scaling_data=True)
        gas.use_machine_learning_model_for_classification(model=None) #MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=1000, activation='relu', solver='adam', random_state=1))

        gas.get_pred_results_df()  


# Visualization (not yet fully organized)

## plotly

In [ ]:
# gcc.make_auto_annot()
# gas.point_index_to_plot = gcc.auto_annot['starting_point_index'].iloc[10]

In [ ]:
gas.traj_points = gas.traj_points_df.values
gas.polar_plots_kwargs['traj_points_to_plot'] = gas.traj_points[gas.indices_test]

In [ ]:
for i in [1, 3, 7, 8, 14]:
    print(i)
    current_plotly_polar_plot_kargs, all_ff_dict = plotly_utils_polar.prepare_to_make_one_plotly_polar_plot(i, gas.polar_plots_kwargs, gas.point_index_to_plot, gas.GUAT_joined_ff_info, gas.all_traj_feature_names, 
                                                                                                            more_ff_df=gas.more_ff_df, trajectory_features=gc_kwargs['trajectory_features'])

    fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs,
                                                        symbol='group', color='time_since_last_vis', 
                                                        columns_for_annotation=['subgroup', 'curv_diff', 'time_till_next_visible', 'time_since_last_vis'],)
                                                        #columns_for_annotation=['subgroup', 'ff_number', 'group', 'curv_diff'],)
    print(all_ff_dict['more_monkey_info_for_ff_in_past_or_future'])
    fig.show()

## dash

In [ ]:
from dash import Dash, html, dcc, Input, Output, callback
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.express as px
reload(plotly_utils_polar)
def use_dash(i, gas):
    current_plotly_polar_plot_kargs, all_ff_dict = plotly_utils_polar.prepare_to_make_one_plotly_polar_plot(i, gas.polar_plots_kwargs, gas.point_index_to_plot, gas.GUAT_joined_ff_info, gas.all_traj_feature_names, 
                                                                                            more_ff_df=gas.more_ff_df, trajectory_features=gas.trajectory_features,)
    fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs, columns_for_annotation=['subgroup', 'curv_diff', 'time_since_last_vis', 'time_till_next_visible'], 
                                                                            additional_customdata_columns=['ff_number'], color='group')
    initial_hover_data = current_plotly_polar_plot_kargs['ff_df'][customdata_columns].iloc[0].values
    ff_number_index = np.where(np.array(customdata_columns) == 'ff_number')[0][0]

    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    df = all_ff_dict['combined_ff_df'].copy()

    app = Dash(__name__, external_stylesheets=external_stylesheets)

    app.layout = html.Div([

        html.Div([
            dcc.Graph(
                id='main_graph',
                figure = fig,
                hoverData={'points': [{'customdata': initial_hover_data}]} #['Original-Present', 0]}]}
            )
        ], style={'width': '50%', 'display': 'inline-block', 'padding': '0 0'}),
        html.Div([
            dcc.Graph(id='ff_history'),
        ], style={'display': 'inline-block', 'width': '50%'}),
    ])



    @callback(
        Output('ff_history', 'figure'),
        Input('main_graph', 'hoverData'))
    def update_ff_history(hoverData):
        print('hoverData:', hoverData)
        print('hoverData[points][0]:', hoverData['points'][0])
        current_ff_number = hoverData['points'][0]['customdata'][ff_number_index]
        # if current_ff_number is not int
        if not isinstance(current_ff_number, int):
            raise PreventUpdate("No update was made because current_ff_number is not an integer.")
        else:
            dff = df[df['ff_number'] == current_ff_number]
            current_group = dff['group'].iloc[0]
            symbol_maps_for_groups = {'Original': 'circle-dot', 'Alternative': 'star-dot', 'More': 'diamond-dot'}

            title = '<b>{}</b>'.format(current_ff_number)
            current_plotly_polar_plot_kargs_temp = current_plotly_polar_plot_kargs.copy()
            current_plotly_polar_plot_kargs_temp['ff_df'] = dff.copy()
            fig, customdata_columns = plotly_utils_polar.make_one_plotly_polar_plot(**current_plotly_polar_plot_kargs_temp,
                                                                columns_for_annotation=['subgroup', 'ff_distance', 'ff_angle', 'curv_diff', 'time_since_last_vis', 'time_till_next_visible'],
                                                                add_colorbar=True, add_legend=True, add_real_and_predicted_labels=False, size=None,
                                                                size_max=15, symbol='time_label', symbol_map={'Present':symbol_maps_for_groups[current_group]})
            return fig

    app.run(debug=True)
    return 

In [ ]:
use_dash(1, gas)

In [ ]:
use_dash(3, gas)

In [ ]:
use_dash(7, gas)

## Other important distributions

### histplot of durations between stops

In [ ]:
# plot the duration of the three stops
diff_stop_time = GUAT_cluster_df['last_stop_time'] - GUAT_cluster_df['first_stop_time']
sns.histplot(diff_stop_time, bins=20)
plt.show()


# make a line plot of 3 points on x-axis, with y-axis being time since first stop


### histplot of time_till_next_visible

In [ ]:
## histplot of durations between stops

In [ ]:
sns.histplot(gcc.GUAT_cur_ff_info['time_till_next_visible'], bins=20, stat='probability')
plt.title('time till next visible')
plt.show()

### curvature upper lower bounds

In [ ]:
# see clean_curvature_info function

In [ ]:
for column in ['curv_of_traj', 'curvature_lower_bound', 'curvature_upper_bound', 'opt_arc_curv']:
    sns.boxplot(gas.curvature_df[column].values, orient='h')
    plt.title(column)
    plt.show()
    
   
    sns.histplot(gas.curvature_df[column].values, bins=50)
    plt.title(column)
    plt.show()
    